In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os
import scipy as sc
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error as MAE

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


С прошлой недели я ушёл с результатом 19.5: xgboost без тюнинга, где использовались:
        - дневные и часовые лаги
        - скользящее среднее на 6 часов,
        - гармонические фичи для сезонностей
        - дни недели
На этой неделе попробовал пообучаться на чём-то более быстром, чем xgboost (подбор признаков), затем взял параметры для xgboost и посчитал две версии. Здесь показана версия на ошибку 16.7, потом я добавил ещё скользящих сумм, это считалось 12 часов на достаточно мощном компьютере, и это дало результат 16.19. Но такое время не приемлимо.
Что было добавлено:
        - знаменательные даты (не все давали прирост)
        - ещё скользящих средник
        - погода (парсинг погоды в соседнем ноутбуке)
       

In [2]:
# все данные
df = pd.read_csv('all_trip_data.csv', index_col='time')
df.index = pd.to_datetime(df.index)

In [3]:
# sin-cos
def make_fourier_features(data, num_end, f_ind='week_'):
    length = data.shape[0]   
    for i in range(1, num_end + 1):
        sin = "s_" + f_ind + str(i)
        cos = "c_" + f_ind + str(i)
        data[sin] = np.sin(2*np.pi*i*np.arange(1, length+1)/168.0)
        data[cos] = np.cos(2*np.pi*i*np.arange(1, length+1)/168.0)

# day_of_week
def make_week_day_features(data):
    data['week']= pd.DatetimeIndex(data.index).dayofweek
    data['day']= pd.DatetimeIndex(data.index).day
    data['hour']= pd.DatetimeIndex(data.index).hour


In [4]:
# данные для мая
R = 102
H = 739
Q_may = 0
error_for_month = 1.0/(R*H*6)

In [5]:
# base features
def make_features(region_df, offset, degree, K_d, K_h):
     
    # 6-hours-window
    region_df['last_6_hours_sum'] = region_df['val'].rolling(6).sum().fillna(0)
    region_df['last_6_hours_mean'] = region_df['last_6_hours_sum']/6
    
    # fourier components
    make_fourier_features(region_df, degree)
    
    # weekday dummy components
    make_week_day_features(region_df)
    
    # day lags
    for day_lag in range(1, K_d):
        region_df['day_lag_'+str(day_lag)] = [0]*offset + region_df[offset-24*day_lag:-24*day_lag]['val'].values.tolist()
        
    # hour lags
    for hour_lag in range(1, K_h):
        region_df['hour_lag_'+str(hour_lag)] = [0]*offset + region_df[offset-hour_lag:-hour_lag]['val'].values.tolist()
        
    return region_df

In [6]:
# Функция подсчёта ошибки на конкретном регионе (так лучше, чем считать все, как было сделано на 5ой неделе)
# с базовыми признаками
def train_predict(data, train_time_end, test_time_end, region, denom, f_make_features,
                       pred_start='2016-05-01', pred_end='2016-05-31', degree=49,
                       K_d=2, K_h=8):
    
    region_df = pd.DataFrame(data.loc[:test_time_end][str(region)].values, columns=['val'],
                            index = data.loc[:test_time_end].index)
       
    abs_err_sum = 0
    offset = max(24*K_d, K_h, 12)
    
    # базовые признаки
    region_df = f_make_features(region_df, offset, degree, K_d, K_h)
             
    for i in range(6):
        # Training 
        train_length = data.loc[:train_time_end].shape[0] - i
        regressor = ElasticNet(alpha=0.01, l1_ratio=0.4)
        
        X_train = region_df.iloc[offset:train_length].drop(['val'], axis = 1)
        y_train = region_df.iloc[offset + i + 1:train_length + i + 1].val
        regressor.fit(X_train,y_train)

        # predictions
        X_test = region_df[train_time_end:test_time_end].drop(['val'], axis=1)
        prediction = regressor.predict(X_test)
        prediction[prediction < 0] = 0.0
        
        # 6 hours combinations
        begin_hour = pred_start +' 0' + str(0 + i)+':00:00'
        end_hour = pred_end + ' ' + str(18 + i) + ':00:00'
        
        error = denom * np.abs(prediction - data.loc[begin_hour:end_hour][str(region)].values)
        abs_err_sum += error.sum()
        
#     print(region, abs_err_sum)    
    del region_df
    return abs_err_sum

In [223]:
%%time
# проверка на базовых фичах 
err = train_predict(df.loc[:'2016-05-31 23:00:00'], 
                    train_time_end, test_time_end, 1075, error_for_month, make_features)

print (err)
err = train_predict(df.loc[:'2016-05-31 23:00:00'], 
                    train_time_end, test_time_end, 1232, error_for_month, make_features)
print (err)

1075 0.106891870464
1232 0.65010569182
Wall time: 7.81 s


In [7]:
# Проверка для всех регионов
def calculate_error(data_time_end, train_time_end, test_time_end, feature_generation_function):
    Q_error = 0
    for region in df.columns:
        res = train_predict(df.loc[:data_time_end], 
                            train_time_end,  test_time_end, region, 
                            error_for_month, feature_generation_function)
        Q_error += res
    print ("\n", Q_error)

In [284]:
%%time
# для мая
train_time_end = '2016-04-30 23:00:00'
test_time_end = '2016-05-31 17:00:00'
calculate_error('2016-05-31 23:00:00',train_time_end, test_time_end, make_features)


 18.5500819187


Отлично, попробуем добавить праздники

In [8]:
# праздники учитывал только для своего промежутка полученных данных: 08-15 : 06-16,
def make_features_with_holidays(region_df, offset, degree, K_d, K_h):
     
    region_df = make_features(region_df, offset, degree, K_d, K_h)
    
    region_df['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in region_df.index]
    region_df['chistmas'] = [1 if date.month == 12 and date.day == 25 else 0 for date in region_df.index] 
    region_df['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in region_df.index]
    region_df['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in region_df.index]
    region_df['9/11'] = [1 if date.month == 9 and date.day == 11 else 0 for date in region_df.index]

    # not common holiday date
    # thanks_giving_day : last thursday(November)
    region_df['thanks_giving_day'] = [1 if date.month == 11 and date.day == 26 
                                      else 0 for date in region_df.index]
    # president's day: third fmonday(February)
    region_df['president_day'] = [1 if date.month == 2 and ((date.year == 2015 and date.day == 16) or
                                       (date.year == 2016 and date.day == 15))
                                  else 0 for date in region_df.index]
    
    # memorial day last monday(May) - decrease quality
#     region_df['memorial_day'] = [1 if (date.year == 2016 and date.day == 30 and date.month == 5) or
#                                        (date.year == 2016 and date.day == 15)
#                                   else 0 for date in region_df.index]
   
    return region_df

In [232]:
%%time
# base features + holidays 
err = train_predict(df.loc[:'2016-05-31 23:00:00'],
                    train_time_end, test_time_end, 1075, error_for_month, make_features_with_holidays)

print (err)
err = train_predict(df.loc[:'2016-05-31 23:00:00'],
                    train_time_end, test_time_end, 1232, error_for_month, make_features_with_holidays)
print (err)

1075 0.106736923361
1232 0.649427030493
Wall time: 8.73 s


In [288]:
%%time
# для мая
train_time_end = '2016-04-30 23:00:00'
test_time_end = '2016-05-31 17:00:00'
calculate_error('2016-05-31 23:00:00',train_time_end, test_time_end, make_features_with_holidays)


 18.5287810451
Wall time: 5min 41s


Праздники помогли слегка снизить ошибку, но незначительно. Думаю проблема в небольшом датасете и числе праздников, многие значительные праздники не обозначены, т.к вообще не встречаются в рассмотренных данных

### Weather history
Добавим признаки, которые относятся к погоде. Погода была получена парсингом сайта https://www.wunderground.com
Код, получающий данные приведён в соседнем ноутбуке

In [9]:
weather_data = pd.read_csv('weather.csv',names = ['time','temperature','wind', 'old_time'],
                           index_col = 'time')

# приводим в порядок типы данных
weather_data.index = pd.to_datetime(weather_data.index)
weather_data = weather_data[['temperature','wind']]['2015-08-01 00:00:00':'2016-06-30 17:00:00']
weather_data['temperature'] = (weather_data['temperature']).astype(float)
weather_data['wind'] = (weather_data['wind']).astype(float)

In [10]:
# праздники учитывал только для своего промежутка полученных данных: 08-15 : 06-16,
def make_features_with_holidays_and_weather(region_df, offset, degree, K_d, K_h):
     
    region_df = make_features_with_holidays(region_df, offset, degree, K_d, K_h)
    
    region_df['temperature'] = weather_data['temperature'][region_df.index]
    region_df['wind'] = weather_data['wind'][region_df.index]
   
    return region_df

In [292]:
%%time
# base features + holidays + weather
err = train_predict(df.loc[:'2016-05-31 23:00:00'],
                    train_time_end, test_time_end, 1075, 
                    error_for_month, make_features_with_holidays_and_weather)
print (err)

err = train_predict(df.loc[:'2016-05-31 23:00:00'],
                    train_time_end, test_time_end, 1232, 
                    error_for_month, make_features_with_holidays_and_weather)
print (err)

0.107422825117
0.645917246877
Wall time: 8.92 s


In [294]:
%%time
# для мая
train_time_end = '2016-04-30 23:00:00'
test_time_end = '2016-05-31 17:00:00'
calculate_error('2016-05-31 23:00:00',train_time_end, test_time_end, make_features_with_holidays_and_weather)


 18.5093851825
Wall time: 5min 49s


Ошибка снизилась ещё чуть-чуть, но я ожидал больше.
Попробуем вернуться к xgboost


In [16]:
def train_predict_xgboost(data, train_time_end, test_time_end, region, denom, f_make_features,
                          pred_start='2016-05-01', pred_end='2016-05-31', degree=49,
                          K_d=2, K_h=8):
    
    region_df = pd.DataFrame(data.loc[:test_time_end][str(region)].values, columns=['val'],
                            index = data.loc[:test_time_end].index)
       
    abs_err_sum = 0
    offset = max(24*K_d, K_h, 12)
    
    # create features
    region_df = f_make_features(region_df, offset, degree, K_d, K_h)
    
    for i in range(6):
        # Training 
        train_length = data.loc[:train_time_end].shape[0] - i
        X_train = region_df.iloc[offset:train_length].drop(['val'], axis=1)
        y_train = region_df.iloc[offset + i + 1:train_length + i + 1].val
        xgb_train = xgb.DMatrix(X_train, y_train)
 
        params={
            'max_depth': 6, 
            'eta': 0.05, 
            'colsample_bytree': 1,
            "min_child_weight": 6, 
            'subsample': 0.8,
            'gamma': 1, # L2
            'alpha': 0.01, # L1
            'objective': "reg:linear",
            'eval_metric': 'mae',
            'nthread': -1
        }
        evals = [(xgb_train, 'train')]
        bst = xgb.train(params, xgb_train, num_boost_round=200, early_stopping_rounds=6, 
                        evals=evals, verbose_eval=False)
       
        # predictions
        X_test= xgb.DMatrix(region_df[train_time_end:].drop(['val'], axis=1))
        prediction = bst.predict(X_test)
        
        prediction[prediction < 0] = 0.0
        
        # 6 hours combinations
        begin_hour = pred_start +' 0' + str(0 + i)+':00:00'
        end_hour = pred_end + ' ' + str(18 + i) + ':00:00'
        
        error = denom * np.abs(prediction - data.loc[begin_hour:end_hour][str(region)].values)
        abs_err_sum += error.sum()
 
    del region_df
    return abs_err_sum

In [15]:
%%time
train_time_end = '2016-04-30 23:00:00'
test_time_end = '2016-05-31 17:00:00'
# base features + holidays + weather + xgboost
err = train_predict_xgboost(df.loc[:'2016-05-31 23:00:00'],
                    train_time_end, test_time_end, 1075, 
                    error_for_month, make_features_with_holidays_and_weather)
print (err)

0.405099590352
Wall time: 18.6 s


Похоже есть смысл обучать xgboost. Посчитаем весь май

In [14]:
R = 102
H = 739
error_for_month = 1.0/(R*H*6)

In [17]:
%%time
# для мая
Q_error = 0
train_time_end = '2016-04-30 23:00:00'
test_time_end = '2016-05-31 17:00:00'
for region in df.columns:   
    
    res = train_predict_xgboost(df.loc[:'2016-05-31 23:00:00'], 
                            train_time_end,  test_time_end, region, 
                            error_for_month, make_features_with_holidays_and_weather)
    print(region,res)
    Q_error += res

1075 0.100425263714
1076 0.160887838109
1077 0.112556216103
1125 0.0967478421516
1126 0.18992078782
1127 0.248396661932
1128 0.291737224282
1129 0.319642570408
1130 0.413333063862
1131 0.198466692023
1132 0.128697815978
1172 0.0290549653696
1173 0.0663604606156
1174 0.0603316334289
1175 0.039505447759
1176 0.0440218878337
1177 0.310804390151
1178 0.389253581169
1179 0.382649729757
1180 0.444021616872
1181 0.752831062092
1182 0.419635729959
1183 0.193833000149
1184 0.0428766428568
1221 0.0235356829208
1222 0.0330880561974
1223 0.0525771649062
1224 0.0567362468211
1225 0.022880493873
1227 0.20221219919
1228 0.341856981136
1229 0.404440440531
1230 0.526570769752
1231 0.608159369811
1232 0.618818516669
1233 0.491892449983
1234 0.32573999373
1235 0.15994266704
1272 0.027211810034
1273 0.0511183762341
1274 0.0251025831961
1278 0.0354737436074
1279 0.0568600081418
1280 0.194219609723
1281 0.572859873112
1282 0.719514249194
1283 0.272658059023
1284 0.119588704831
1285 0.25303776754
1286 0.1996

![title](fun.png)

Неплохой результат (съехал, увы, было 16.99) за страшное время. Сделаем то же самое для июня с сабмитом

In [77]:
def train_predict_xgboost2(data, train_time_end, test_time_end, region, denom, f_make_features,
                          pred_start='2016-05-01', pred_end='2016-05-31', degree=49,
                          K_d=2, K_h=8):
    
    region_df = pd.DataFrame(data.loc[:test_time_end][str(region)].values, columns=['val'],
                            index = data.loc[:test_time_end].index)
       
    abs_err_sum = 0
    offset = max(24*K_d, K_h, 12)
    
    # create features
    region_df = f_make_features(region_df, offset, degree, K_d, K_h)
    
    # submit
    submit_ids = []
    submit_preds = []
    
    for i in range(6):
        # Training 
        train_length = data.loc[:train_time_end].shape[0] - i
        X_train = region_df.iloc[offset:train_length].drop(['val'], axis=1)
        y_train = region_df.iloc[offset + i + 1:train_length + i + 1].val
        xgb_train = xgb.DMatrix(X_train, y_train)
 
        params={
            'max_depth': 6, 
            'eta': 0.05, 
            'colsample_bytree': 1,
            "min_child_weight": 6, 
            'subsample': 0.8,
            'gamma': 1, # L2
            'alpha': 0.01, # L1
            'objective': "reg:linear",
            'eval_metric': 'mae',
            'nthread': -1
        }
        evals = [(xgb_train, 'train')]
        bst = xgb.train(params, xgb_train, num_boost_round=200, early_stopping_rounds=6, 
                        evals=evals, verbose_eval=False)
       
        # predictions
        X_test= xgb.DMatrix(region_df[train_time_end:].drop(['val'], axis=1))
        prediction = bst.predict(X_test)
        
        prediction[prediction < 0] = 0.0
        
        # 6 hours combinations
        begin_hour = pred_start +' 0' + str(0 + i)+':00:00'
        end_hour = pred_end + ' ' + str(18 + i) + ':00:00'
        
        error = denom * np.abs(prediction - data.loc[begin_hour:end_hour][str(region)].values)
        abs_err_sum += error.sum()
        
        
        indexes = region_df[train_time_end:].index 
        submit_ids.append((pd.Series([str(region)] * indexes.size) + '_' 
                           + list(map(lambda x: x.strftime('%Y-%m-%d'), indexes)) + '_' \
                            + list(map(lambda x: str(x.hour) + '_' + str(i+1), indexes))).values)
        submit_preds.append(prediction)
 
    del region_df
    return abs_err_sum,submit_ids,submit_preds

In [57]:
%%time

# проверка
train_time_end = '2016-04-30 23:00:00'
test_time_end = '2016-05-31 17:00:00'
# base features + holidays + weather + xgboost
err, submit_ids,submit_preds = train_predict_xgboost2(df.loc[:'2016-05-31 23:00:00'],
                                    train_time_end, test_time_end, 1075, 
                                    error_for_month, make_features_with_holidays_and_weather)
print (err)

0.103796181657
Wall time: 2min 20s


In [78]:
# параметры для июня
R = 102
H = 715
error_for_month = 1.0/(R*H*6)
Q_error = 0
submit_ids = []
submit_preds = []

train_time_end = '2016-05-31 23:00:00'
test_time_end = '2016-06-30 17:00:00'

In [80]:
%%time
# ещё поднимем K_d до 7 (дневные лаги)
for region in df.columns:
    err, ids, pred = train_predict_xgboost2(df.loc[:'2016-06-30 23:00:00'], 
                                           train_time_end, test_time_end , region, error_for_month, 
                                           make_features_with_holidays_and_weather,
                                           '2016-06-01', '2016-06-30', degree=49, K_d=7, K_h=8)
     
    Q_error += err
    print ('region: %s, error: %f, current_error: %f' %(region, err, Q_error))
    submit_ids.append(ids)
    submit_preds.append(pred)

region: 1075, error: 0.109294, current_error: 0.109294
region: 1076, error: 0.147705, current_error: 0.256999
region: 1077, error: 0.121915, current_error: 0.378914
region: 1125, error: 0.087326, current_error: 0.466240
region: 1126, error: 0.196146, current_error: 0.662386
region: 1127, error: 0.253442, current_error: 0.915828
region: 1128, error: 0.280989, current_error: 1.196817
region: 1129, error: 0.319057, current_error: 1.515874
region: 1130, error: 0.414303, current_error: 1.930178
region: 1131, error: 0.187232, current_error: 2.117410
region: 1132, error: 0.124003, current_error: 2.241413
region: 1172, error: 0.020459, current_error: 2.261872
region: 1173, error: 0.051568, current_error: 2.313439
region: 1174, error: 0.061087, current_error: 2.374527
region: 1175, error: 0.036382, current_error: 2.410909
region: 1176, error: 0.041876, current_error: 2.452785
region: 1177, error: 0.282662, current_error: 2.735447
region: 1178, error: 0.471888, current_error: 3.207335
region: 11

In [82]:
pred_df = pd.DataFrame(np.array(submit_preds).ravel(), index=np.array(submit_ids).ravel(), columns=['y'])
pred_df.index.name = 'id'
pred_df.to_csv("w6.csv", sep=',')

![title](submit.png)

Ссылка на лучший сабмит, считался 12 часов: https://inclass.kaggle.com/c/yellowtaxi/leaderboard?submissionId=5039542
Улучшение составило ~3.5 относительно прошлой недели